In [1]:
# Text Based Emotion Recog using NLP Vect

In [2]:
!pip install onednn-cpu-gomp

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [26]:

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'


In [6]:
import tensorflow
print(tensorflow.__version__)

import keras
print(keras.__version__)

2.12.0
2.12.0


In [7]:
#install the NLTK (Natural Language Toolkit)
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [8]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
train=pd.read_table('train.txt', delimiter = ';', header=None, )
val=pd.read_table('val.txt', delimiter = ';', header=None, )
test=pd.read_table('test.txt', delimiter = ';', header=None, )

data = pd.concat([train ,  val , test])
data.columns = ["text", "label"]



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [9]:
data.shape

(20000, 2)

In [10]:
data.isna().any(axis=1).sum()

0

In [11]:
#text preprocessing
ps = PorterStemmer()

def preprocess(line):
    review = re.sub('[^a-zA-Z]', ' ', line) #leave only characters from a to z
    review = review.lower() #lower the text
    review = review.split() #turn string into list of words
    #apply Stemming 
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #delete stop words like I, and ,OR   review = ' '.join(review)
    #trun list into sentences
    return " ".join(review)


In [12]:
#install the NLTK (Natural Language Toolkit)

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/u190492/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
data['text']=data['text'].apply(lambda x: preprocess(x))

In [14]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
data['N_label'] = label_encoder.fit_transform(data['label'])


In [15]:
data['text']

0                                       didnt feel humili
1       go feel hopeless damn hope around someon care ...
2                    im grab minut post feel greedi wrong
3          ever feel nostalg fireplac know still properti
4                                            feel grouchi
                              ...                        
1995    keep feel like someon unkind wrong think get b...
1996              im feel littl cranki neg doctor appoint
1997                feel use peopl give great feel achiev
1998    im feel comfort derbi feel though start step s...
1999    feel weird meet w peopl text like dont talk fa...
Name: text, Length: 20000, dtype: object

In [16]:
# Creating the Bag of Words model by applying Countvectorizer -convert textual data to numerical data


from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,ngram_range=(1,3))#example: the course was long-> [the,the course,the course was,course, course was, course was long,...]

data_cv = cv.fit_transform(data['text']).toarray()




In [17]:
data_cv

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
#X_train, X_test, y_train, y_test=data_cv,test_cv,train['N_label'],test['N_label']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(data_cv, data['N_label'], test_size=0.25, random_state=42)


In [19]:
# first neural network with keras tutorial
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# load the dataset
# split into input (X) and output (y) variables
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=10, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 1.0831 - accuracy: 0.5949
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3234 - accuracy: 0.8983
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1604 - accuracy: 0.9486
Epoch 4/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1020 - accuracy: 0.9681
Epoch 5/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0683 - accuracy: 0.9783
Epoch 6/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0494 - accuracy: 0.9843
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0375 - accuracy: 0.9888
Epoch 8/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0294 - accuracy: 0.9913
Epoch 9/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0226 - accuracy: 0.9929
Epoch 10/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0125 

In [20]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

157/157 [==============================] - 0s 3ms/step - loss: 0.7810 - accuracy: 0.8532
Accuracy: 85.32


In [33]:
text='Hello World !'
text=preprocess(text)
array = cv.transform([text]).toarray()
pred = model.predict(array)
a=np.argmax(pred, axis=1)
label_encoder.inverse_transform(a)[0]

1/1 [==============================] - 0s 18ms/step


'anger'

In [22]:
import tensorflow as tf

In [27]:
tf.keras.models.save_model(model,'my_modelnew.h5')


In [28]:
import pickle
pickle.dump(label_encoder, open('encoder.pkl', 'wb'))
pickle.dump(cv, open('CountVectorizer.pkl', 'wb'))
